In [137]:
from bs4 import BeautifulSoup
import urllib.request
import xml.etree.ElementTree as ET
import geopandas as gpd
from shapely.geometry import Point, Polygon
import pandas as pd

# Sentinel 1 

### Extract KML data

In [138]:
# Import Sentinel 1 acquistion segments
s1_url = 'https://sentinels.copernicus.eu/web/sentinel/missions/sentinel-1/observation-scenario/acquisition-segments'
s1_r = urllib.request.urlopen(s1_url).read()
s1_soup = BeautifulSoup(s1_r, "html.parser")

# Find all links to KML files that end with .kml or contain "s1a_mp_user" (case-insensitive)
s1_kml_links = s1_soup.find_all('a', href=lambda href: href.endswith('.kml') or 's1a_mp_user' in href.lower())

# Create a list to store the kml_content
s1_kml_contents = []

# Iterate over the links
for kml_link in s1_kml_links:
    # Retrieve the URL of each KML file
    s1_kml_url = 'https://sentinels.copernicus.eu' + kml_link['href']

    # Open the KML file
    s1_kml_response = urllib.request.urlopen(s1_kml_url)

    # Read the content of the KML file
    s1_kml_content = s1_kml_response.read()

    # Append the kml_content to the list
    s1_kml_contents.append(s1_kml_content)

# Write the sentinel_1.kml file
s1_kml_file = 'sentinel_1.kml'
with open(s1_kml_file, 'wb') as file:
    for s1_kml_content in s1_kml_contents:
        file.write(s1_kml_content)


### Parse KML and convert into DataFrame

In [139]:
# Parse the KML content
s1_root = ET.fromstring(s1_kml_content)

# Define the XML namespace
s1_namespace = {'kml': 'http://www.opengis.net/kml/2.2'}

# Extract latitude, longitude, time, mode and polarisation 
s1_data = [] # will create a list of dictionaries. 
# Each dictionary in the list represents an observation, and the keys of the dictionary become columns in the DataFrame.
for placemark in s1_root.findall('.//kml:Placemark', s1_namespace):
    coordinates = placemark.find('.//kml:coordinates', s1_namespace).text.strip()
    time_start = placemark.find('.//kml:begin', s1_namespace).text
    time_end = placemark.find('.//kml:end', s1_namespace).text
    mode = placemark.find('.//kml:Data[@name="Mode"]/kml:value', s1_namespace).text
    polarisation = placemark.find('.//kml:Data[@name="Polarisation"]/kml:value', s1_namespace).text
    
    
    # Append the data to the list
    s1_data.append({'coordinates': coordinates, 'time_start': time_start, 'time_end': time_end,
                 'mode': mode, 'polarisation': polarisation})

# Convert the data to a DataFrame
s1_df = pd.DataFrame(s1_data)

### Transform to geospatial data

In [140]:
# Create polygon function
def create_polygon(coords):
    points = []
    for coord in coords.split():
        lon, lat, _ = coord.split(',')
        points.append((float(lon), float(lat)))
    return Polygon(points)

# Create geometry column 
s1_df['geometry'] = s1_df['coordinates'].apply(create_polygon)

# Create a GeoDataFrame with Polygon geometry
s1_gdf = gpd.GeoDataFrame(s1_df, geometry='geometry')

### Format GeoDataFrame

In [141]:
# Apply the filter for mode=='IW' and polarisation=='DV'
s1_gdf = s1_gdf[(s1_gdf['mode'] == 'IW') & (s1_gdf['polarisation'] == 'DV')]

# Manipulate columns
s1_gdf = s1_gdf.drop('coordinates', axis=1)
s1_gdf = s1_gdf.drop('polarisation', axis=1)
s1_gdf = s1_gdf.drop('mode', axis=1)
s1_gdf['satellite'] = 'Sentinel 1'


,time_start,time_end,geometry,satellite
0,2022-01-14T16:15:52,2022-01-14T16:18:38,"POLYGON ((32.42674 -18.55535, 30.99000 -12.564...",Sentinel 1
1,2022-01-14T16:19:41,2022-01-14T16:21:04,"POLYGON ((29.23835 -4.74951, 28.16703 0.25199,...",Sentinel 1
2,2022-01-14T16:26:32,2022-01-14T16:35:49,"POLYGON ((24.05284 20.10057, 23.29609 23.69772...",Sentinel 1
4,2022-01-14T16:38:21,2022-01-14T16:40:50,"POLYGON ((11.87317 62.61987, 5.81056 71.35454,...",Sentinel 1
8,2022-01-14T18:09:10,2022-01-14T18:17:04,"POLYGON ((-3.69491 34.15477, -6.77342 46.29509...",Sentinel 1
...,...,...,...,...
1665,2022-02-03T15:23:49,2022-02-03T15:29:59,"POLYGON ((38.08064 31.37876, 34.79872 44.75346...",Sentinel 1
1666,2022-02-03T15:31:08,2022-02-03T15:31:19,"POLYGON ((30.57200 57.66933, 30.30357 58.32536...",Sentinel 1
1668,2022-02-03T15:33:53,2022-02-03T15:35:44,"POLYGON ((25.45786 67.45300, 19.51202 73.85542...",Sentinel 1
1671,2022-02-03T16:47:18,2022-02-03T16:51:05,"POLYGON ((25.57412 -23.88827, 25.35996 -23.085...",Sentinel 1


# Sentinel 2 

In [142]:
# Import Sentinel 2 acquistion plans
s2_url = 'https://sentinels.copernicus.eu/web/sentinel/missions/sentinel-2/acquisition-plans'
s2_r = urllib.request.urlopen(s2_url).read()
s2_soup = BeautifulSoup(s2_r, "html.parser")

### Exract Sentinel 2A KML data

In [143]:
# Find all Sentinel 2A KML files
s2a_kml_links = s2_soup.find_all('a', href=lambda href:  's2a_mp_acq__kml' in href.lower())

# Create a list to store the kml_content
s2a_kml_contents = []

# Iterate over the links
for kml_link in s2a_kml_links:
    # Retrieve the URL of each KML file
    s2a_kml_url = 'https://sentinels.copernicus.eu' + kml_link['href']

    # Open the KML file
    s2a_kml_response = urllib.request.urlopen(s2a_kml_url)

    # Read the content of the KML file
    s2a_kml_content = s2a_kml_response.read()

    # Append the kml_content to the list
    s2a_kml_contents.append(s2a_kml_content)

# Write the sentinel_2a.kml file
s2a_kml_file = 'sentinel_2a.kml'
with open(s2a_kml_file, 'wb') as file:
    for s2a_kml_content in s2a_kml_contents:
        file.write(s2a_kml_content)

# Parse the KML content
s2a_root = ET.fromstring(s2a_kml_content)

# Define the XML namespace
s2a_namespace = {'kml': 'http://www.opengis.net/kml/2.2'}

# Extract latitude, longitude, time, mode and polarisation 
s2a_data = [] # will create a list of dictionaries. 
# Each dictionary in the list represents an observation, and the keys of the dictionary become columns in the DataFrame.
for placemark in s2a_root.findall('.//kml:Placemark', s2a_namespace):
    coordinates = placemark.find('.//kml:coordinates', s2a_namespace).text.strip()
    time_start = placemark.find('.//kml:begin', s2a_namespace).text
    time_end = placemark.find('.//kml:end', s2a_namespace).text
    mode = placemark.find('.//kml:Data[@name="Mode"]/kml:value', s2a_namespace).text
   
    
    # Append the data to the list
    s2a_data.append({'coordinates': coordinates, 'time_start': time_start, 'time_end': time_end,
                 'mode': mode})

# Convert the data to a DataFrame
s2a_df = pd.DataFrame(s2a_data)

# Create geometry column 
s2a_df['geometry'] = s2a_df['coordinates'].apply(create_polygon)

# Create a GeoDataFrame with Polygon geometry
s2a_gdf = gpd.GeoDataFrame(s2a_df, geometry='geometry')

# Apply the filter for mode=='NOBS'
s2a_gdf = s2a_gdf[(s2a_gdf['mode'] == 'NOBS')]

# Format GeoDataFrame
s2a_gdf = s2a_gdf.drop('coordinates', axis=1)
s2a_gdf = s2a_gdf.drop('mode', axis=1)
s2a_gdf['satellite'] = 'Sentinel 2A'


,time_start,time_end,geometry,satellite
38,2023-06-29T12:16:28.516,2023-06-29T12:16:46.556,"POLYGON ((-37.78600 -52.31643, -39.84274 -51.9...",Sentinel 2A
39,2023-06-29T13:17:09.430,2023-06-29T13:24:11.566,"POLYGON ((49.40952 81.47978, 49.62026 82.78700...",Sentinel 2A
40,2023-06-29T13:38:30.435,2023-06-29T13:53:32.435,"POLYGON ((-45.02756 13.65629, -46.32848 13.937...",Sentinel 2A
41,2023-06-29T13:56:27.314,2023-06-29T13:57:10.610,"POLYGON ((-61.81858 -49.82582, -63.77340 -49.4...",Sentinel 2A
42,2023-06-29T14:57:51.804,2023-06-29T15:05:30.020,"POLYGON ((24.06645 81.48024, 24.24760 82.78755...",Sentinel 2A
...,...,...,...,...
874,2023-07-17T13:02:42.456,2023-07-17T13:09:30.160,"POLYGON ((-38.50608 -2.28816, -39.77235 -2.005...",Sentinel 2A
875,2023-07-17T14:17:37.140,2023-07-17T14:18:49.300,"POLYGON ((33.27910 81.48002, 33.30941 82.78728...",Sentinel 2A
876,2023-07-17T14:20:17.370,2023-07-17T14:26:03.738,"POLYGON ((-15.98400 77.28091, -20.64118 78.133...",Sentinel 2A
877,2023-07-17T14:29:05.427,2023-07-17T14:29:59.547,"POLYGON ((-50.04809 48.39567, -51.94729 48.728...",Sentinel 2A


### Sentinel 2B

In [144]:
# Find all Sentinel 2B KML files
s2b_kml_links = s2_soup.find_all('a', href=lambda href:  's2b_mp_acq__kml' in href.lower())

# Create a list to store the kml_content
s2b_kml_contents = []

# Iterate over the links
for kml_link in s2b_kml_links:
    # Retrieve the URL of each KML file
    s2b_kml_url = 'https://sentinels.copernicus.eu' + kml_link['href']

    # Open the KML file
    s2b_kml_response = urllib.request.urlopen(s2b_kml_url)

    # Read the content of the KML file
    s2b_kml_content = s2b_kml_response.read()

    # Append the kml_content to the list
    s2b_kml_contents.append(s2b_kml_content)

# Write the sentinel_2b.kml file
s2b_kml_file = 'sentinel_2b.kml'
with open(s2b_kml_file, 'wb') as file:
    for s2b_kml_content in s2b_kml_contents:
        file.write(s2b_kml_content)

# Parse the KML content
s2b_root = ET.fromstring(s2b_kml_content)

# Define the XML namespace
s2b_namespace = {'kml': 'http://www.opengis.net/kml/2.2'}

# Extract latitude, longitude, time, mode and polarisation 
s2b_data = [] # will create a list of dictionaries. 
# Each dictionary in the list represents an observation, and the keys of the dictionary become columns in the DataFrame.
for placemark in s2b_root.findall('.//kml:Placemark', s2b_namespace):
    coordinates = placemark.find('.//kml:coordinates', s2b_namespace).text.strip()
    time_start = placemark.find('.//kml:begin', s2b_namespace).text
    time_end = placemark.find('.//kml:end', s2b_namespace).text
    mode = placemark.find('.//kml:Data[@name="Mode"]/kml:value', s2b_namespace).text
   
    
    # Append the data to the list
    s2b_data.append({'coordinates': coordinates, 'time_start': time_start, 'time_end': time_end,
                 'mode': mode})

# Convert the data to a DataFrame
s2b_df = pd.DataFrame(s2b_data)

# Create geometry column 
s2b_df['geometry'] = s2b_df['coordinates'].apply(create_polygon)

# Create a GeoDataFrame with Polygon geometry
s2b_gdf = gpd.GeoDataFrame(s2b_df, geometry='geometry')

# Apply the filter for mode=='NOBS'
s2b_gdf = s2b_gdf[(s2b_gdf['mode'] == 'NOBS')]

# Format GeoDataFrame
s2b_gdf = s2b_gdf.drop('coordinates', axis=1)
s2b_gdf = s2b_gdf.drop('mode', axis=1)
s2b_gdf['satellite'] = 'Sentinel 2B'

s2b_gdf

,time_start,time_end,geometry,satellite
0,2023-07-06T12:16:41.510,2023-07-06T12:19:49.126,"POLYGON ((64.81570 81.47877, 65.07941 82.78581...",Sentinel 2B
1,2023-07-06T12:20:31.325,2023-07-06T12:21:39.877,"POLYGON ((4.21681 74.15072, 0.06129 74.83934, ...",Sentinel 2B
2,2023-07-06T12:23:31.765,2023-07-06T12:23:49.805,"POLYGON ((-10.08105 64.49324, -12.94320 64.949...",Sentinel 2B
3,2023-07-06T12:25:19.208,2023-07-06T12:25:37.248,"POLYGON ((-14.72131 58.42188, -17.10818 58.813...",Sentinel 2B
4,2023-07-06T12:30:37.441,2023-07-06T12:31:49.601,"POLYGON ((-22.93917 39.96669, -24.58822 40.273...",Sentinel 2B
...,...,...,...,...
849,2023-07-24T12:16:33.362,2023-07-24T12:17:09.442,"POLYGON ((-37.97705 -52.71060, -40.05117 -52.3...",Sentinel 2B
850,2023-07-24T13:17:07.430,2023-07-24T13:24:09.566,"POLYGON ((49.40945 81.47978, 49.62018 82.78700...",Sentinel 2B
851,2023-07-24T13:38:28.435,2023-07-24T13:53:30.435,"POLYGON ((-45.02756 13.65628, -46.32848 13.937...",Sentinel 2B
852,2023-07-24T13:56:25.314,2023-07-24T13:57:08.610,"POLYGON ((-61.81859 -49.82583, -63.77341 -49.4...",Sentinel 2B


# Combine GeoDataFrames

In [145]:
combined_gdf = pd.concat([s1_gdf, s2a_gdf, s2b_gdf], ignore_index=True)
combined_gdf

,time_start,time_end,geometry,satellite
0,2022-01-14T16:15:52,2022-01-14T16:18:38,"POLYGON ((32.42674 -18.55535, 30.99000 -12.564...",Sentinel 1
1,2022-01-14T16:19:41,2022-01-14T16:21:04,"POLYGON ((29.23835 -4.74951, 28.16703 0.25199,...",Sentinel 1
2,2022-01-14T16:26:32,2022-01-14T16:35:49,"POLYGON ((24.05284 20.10057, 23.29609 23.69772...",Sentinel 1
3,2022-01-14T16:38:21,2022-01-14T16:40:50,"POLYGON ((11.87317 62.61987, 5.81056 71.35454,...",Sentinel 1
4,2022-01-14T18:09:10,2022-01-14T18:17:04,"POLYGON ((-3.69491 34.15477, -6.77342 46.29509...",Sentinel 1
...,...,...,...,...
2531,2023-07-24T12:16:33.362,2023-07-24T12:17:09.442,"POLYGON ((-37.97705 -52.71060, -40.05117 -52.3...",Sentinel 2B
2532,2023-07-24T13:17:07.430,2023-07-24T13:24:09.566,"POLYGON ((49.40945 81.47978, 49.62018 82.78700...",Sentinel 2B
2533,2023-07-24T13:38:28.435,2023-07-24T13:53:30.435,"POLYGON ((-45.02756 13.65628, -46.32848 13.937...",Sentinel 2B
2534,2023-07-24T13:56:25.314,2023-07-24T13:57:08.610,"POLYGON ((-61.81859 -49.82583, -63.77341 -49.4...",Sentinel 2B


In [146]:
# Create a Point object for the longitude and latitude you want to check
lon = 24.05284
lat = 20.10057
user_coord = Point(lon, lat)

# Check if the point is contained within any of the polygons in the filtered GeoDataFrame
coord_mask = combined_gdf.geometry.contains(user_coord)

# Get the rows where the point is contained within the polygon(s)
final_gdf = combined_gdf[coord_mask]

final_gdf

,time_start,time_end,geometry,satellite
105,2022-01-17T04:19:51,2022-01-17T04:24:27,"POLYGON ((24.62157 20.20167, 22.61197 10.47602...",Sentinel 1
204,2022-01-19T16:34:15,2022-01-19T16:37:33,"POLYGON ((22.38979 18.20731, 19.84940 30.11141...",Sentinel 1
608,2022-01-29T04:19:51,2022-01-29T04:24:27,"POLYGON ((24.62157 20.20167, 22.61197 10.47602...",Sentinel 1
712,2022-01-31T16:34:15,2022-01-31T16:37:33,"POLYGON ((22.38979 18.20731, 19.84940 30.11141...",Sentinel 1
1253,2023-07-08T08:45:55.181,2023-07-08T09:18:12.677,"POLYGON ((101.87511 81.21880, 99.48278 82.4818...",Sentinel 2A
2003,2023-07-13T08:46:36.868,2023-07-13T09:18:11.068,"POLYGON ((86.65644 80.26418, 82.48459 81.39064...",Sentinel 2B
2478,2023-07-23T08:45:53.181,2023-07-23T09:18:10.677,"POLYGON ((101.87505 81.21880, 99.48271 82.4818...",Sentinel 2B
